### DCGAN with Spectral Normalization and Self Modulation

SpecNorm in the Discriminator, Self Modulation with BatchNorm in the Generator, **Hinge** Loss.
DCGAN Architecture otherwise, no Resblocks added in this particular version

Let's import our torch things. Look at the building_blocks and models python files to figure out what's going underneath the hood!

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from models.building_blocks import ResBlock, UpBlock, DownBlock, ModulatingNet
from models.basic import Basic_Generator, Basic_Discriminator

Loadin' up CIFAR-10

In [ ]:
batch_size_train = 64

def get_indices(dataset,class_name):
    indices =  []
    for i in range(len(dataset.targets)):
        if dataset.targets[i] == class_name:
            indices.append(i)
    return indices

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
idx = get_indices(trainset, 5)

#Use the first one if you only want to produce a particular class
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, num_workers=2, sampler = torch.utils.data.sampler.SubsetRandomSampler(idx))
#trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, num_workers=2, shuffle = True)

In [ ]:
data_loader = iter(trainloader)
(data, target) = next(data_loader)

print(data.shape)
print(data.max(), data.min())
img = np.transpose(data[0], (1, 2, 0))
plt.imshow((img+1)/2)
plt.show()

Settin' up our networks dudes! Pass image size as a tuple when initializing the networks.

Z seems to work fine as a 128 dimensional vector for most datasets.

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

noise_dim = 128
netG = Basic_Generator(image_size = (32,32), noise_dim = noise_dim).to(device)
netD = Basic_Discriminator(image_size = (32,32)).to(device)

if torch.cuda.device_count() > 1:
    netG = nn.DataParallel(netG, list(range(torch.cuda.device_count())))
    netD = nn.DataParallel(netD, list(range(torch.cuda.device_count())))

#Optional orthogonal initialization of weights, does not work with Spectral Normalization!#########
for m in netG.modules():
    if isinstance(m, (nn.Conv2d, nn.Linear, nn.ConvTranspose2d)):
        nn.init.orthogonal_(m.weight)
'''
for m in netD.modules():
    if isinstance(m, (nn.Conv2d, nn.Linear, nn.ConvTranspose2d)):
        nn.init.orthogonal_(m.weight)
'''

#Two Timescale Update Rule
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr = 0.0001, betas = (0.5, 0.999))

The training loop, we're using Hinge Loss btw....

In [ ]:
epochs = 500
disc_steps = 1
path = "./saved_models/"

for epoch in range(epochs):
    for i, (data, target) in enumerate(trainloader):

        #Dealing with the discriminator################################
        #Specify number of disc updates above##############
        for s in range(disc_steps):
            netD.zero_grad()
        
            real_images = data.to(device)
            b_size = real_images.size(0)
        
            output = netD(real_images).view(-1)
            errD_real = torch.mean(F.relu(1 - output))
            errD_real.backward()
            D_x = output.mean().item()

            noise = torch.randn(b_size, noise_dim, device = device)
            fake = netG(noise)

            output = netD(fake.detach()).view(-1)
            errD_fake = torch.mean(F.relu(1 + output))
            errD_fake.backward()
            D_G_z1 = output.mean().item()

            errD = errD_fake + errD_real
            optimizerD.step()
            
        #Dealing with the generator###################################
        netG.zero_grad()

        output = netD(fake).view(-1)
        errG = -torch.mean(output)

        D_G_z2 = output.mean().item()
        errG.backward()
        optimizerG.step()

        if i%100 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                    % (epoch, epochs, i, len(trainloader),
                    errD.item(), errG.item(), D_x, D_G_z1, D_G_z2))
    
    if epoch%100 == 0:
        torch.save(netG.state_dict(), path + "sngan_cifar_G.pth")
        torch.save(netD.state_dict(), path + "sngan_cifar_D.pth")

Lookin' at some results!

In [ ]:
f, a = plt.subplots(4, 8, figsize=(30, 8))
for i in range(8):
  noise = torch.randn(4, noise_dim, device = device)
  with torch.no_grad():
    fake = netG(noise)

  for j in range(4):
      img = fake[j].cpu()
      img = np.transpose(img, (1, 2, 0))
      img = (img+1)/2
      a[j][i].imshow(img)
      a[j][i].axis("off")
    
plt.savefig("CIFCol.png")
plt.show()

plt.imshow(img)
plt.show()